In [1]:
import random

import pandas as pd
import torch

from helper import *
from heterognn import *

The State of the nth node is expressed by 4 real scalars:

v_n -> the voltage at the node
delta_n -> the voltage angle at the node (relative to the slack bus)
p_n -> the net active power flowing into the node
q_n -> the net reactive power flowing into the node


The physical characteristics of the network are described by the power flow equations:

p = P(v, delta, W)
q = Q(v, delta, W)

-> Relate local net power generation with the global state
-> Depends on the topology W of the grid

Electrical grid => Weighted Graph

Nodes in the graph produce/consume power

Edges represent electrical connections between nodes

State Matrix X element of  R(N x 4) => graph signal with 4 features
    => Each row is the state of the corresponding Node

Adjacency Matrix A => sparse matrix to represent the connections of each node, element of R(N x N), Aij = 1 if node i is connected to node j


We use the GNN as a mode phi(X, A, H)

We want to imitate the OPF solution p*
-> We want to minimize a loss L over a dataset T = {{X, p*}}

Objective Function:min arg H of sum over T L(p*,phi(X, A, H)) and we use L = Mean Squared error

Once GNN model phi is trained, we do not need the costly p* from pandapower to make predictions

Input Data X - R^(Nx4): Uniformly sample p_ref and q_ref of each load L with P_L ~ Uniform(0.9 * p_ref, 1.1 * p_ref) and Q_L ~ Uniform(0.9 * q_ref, 1.1 * q_ref)

Pseudocode for X and y in supervised learning:
for each P_L and Q_L:
    Create X with sub-optimal DCOPF results
    Create y with Pandapower calculating p* ACOPF with IPOPT


Spatio-Temporal GNN -> superposition of a gnn with spatial info and a temporal layer (Temporal Conv,LSTM etc.) ??

Bus => Node in GNN








In [8]:
# get lists of simbench codes
all_simbench_codes = sb.collect_all_simbench_codes()
all_simbench_codes

['1-complete_data-mixed-all-0-sw',
 '1-complete_data-mixed-all-1-sw',
 '1-complete_data-mixed-all-2-sw',
 '1-EHVHVMVLV-mixed-all-0-sw',
 '1-EHVHVMVLV-mixed-all-1-sw',
 '1-EHVHVMVLV-mixed-all-2-sw',
 '1-EHVHV-mixed-all-0-sw',
 '1-EHVHV-mixed-all-0-no_sw',
 '1-EHVHV-mixed-all-1-sw',
 '1-EHVHV-mixed-all-1-no_sw',
 '1-EHVHV-mixed-all-2-sw',
 '1-EHVHV-mixed-all-2-no_sw',
 '1-EHVHV-mixed-1-0-sw',
 '1-EHVHV-mixed-1-0-no_sw',
 '1-EHVHV-mixed-1-1-sw',
 '1-EHVHV-mixed-1-1-no_sw',
 '1-EHVHV-mixed-1-2-sw',
 '1-EHVHV-mixed-1-2-no_sw',
 '1-EHVHV-mixed-2-0-sw',
 '1-EHVHV-mixed-2-0-no_sw',
 '1-EHVHV-mixed-2-1-sw',
 '1-EHVHV-mixed-2-1-no_sw',
 '1-EHVHV-mixed-2-2-sw',
 '1-EHVHV-mixed-2-2-no_sw',
 '1-EHV-mixed--0-sw',
 '1-EHV-mixed--0-no_sw',
 '1-EHV-mixed--1-sw',
 '1-EHV-mixed--1-no_sw',
 '1-EHV-mixed--2-sw',
 '1-EHV-mixed--2-no_sw',
 '1-HVMV-mixed-all-0-sw',
 '1-HVMV-mixed-all-0-no_sw',
 '1-HVMV-mixed-all-1-sw',
 '1-HVMV-mixed-all-1-no_sw',
 '1-HVMV-mixed-all-2-sw',
 '1-HVMV-mixed-all-2-no_sw',
 '1-HVM

In [ ]:
net = sb.get_simbench_net('1-HV-mixed--0-no_sw')#'1-HV-mixed--0-no_sw'

In [ ]:
in_channels = 4
hidden_channels = 256
out_channels = 4
activation = "elu"
scaler = StandardScaler()
num_layers = 5
dropout = 0.0
jk = "last"
lr = 0.00001
layer_type = "TransConv"
# torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels)
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN(in_channels, hidden_channels, num_layers, out_channels, dropout=dropout, norm=torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels),jk=jk,layer_type=layer_type, activation=activation)#.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

ordered_dict = torch.load(r"C:\Users\canba\OneDrive\Masaüstü\OPFGNN\code\Models\Supervised\supervisedmodel.pt")

model.load_state_dict(ordered_dict)

In [ ]:
'/'.join(os.path.dirname(os.path.abspath(__file__).split("\\"))) + r"/Models/SelfSupervised/base_model.pt"


In [ ]:
net.bus

In [ ]:
to_json('1-HV-mixed--0-no_sw')

In [ ]:
net = sb.get_simbench_net('1-HV-mixed--0-no_sw')#'1-HV-mixed--0-no_sw'
#Replace all ext_grids but the first one with generators and set the generators to slack= false
ext_grids = [i for i in range(1,len(net.ext_grid.name.values))]
pp.replace_ext_grid_by_gen(net,ext_grids=ext_grids, slack=False)

#TODO: reactive power limits for gens?


#TODO: reactive power limits for sgens?



#NETWORK CONSTRAINTS

#Maximize the branch limits

#max_i_ka = list(net.line.max_i_ka.values)

#for i in range(len(max_i_ka)):
# max_i_ka[i] = max(max_i_ka)



#Maximize line loading percents
max_loading_percent = list(net.line.max_loading_percent.values)
for i in range(len(max_loading_percent)):
    max_loading_percent[i] = 100.0
net.line.max_loading_percent = max_loading_percent

#Maximize trafo loading percent
max_loading_percent = list(net.trafo.max_loading_percent.values)
for i in range(len(max_loading_percent)):
    max_loading_percent[i] = 100.0
net.trafo.max_loading_percent = max_loading_percent

#Maximize trafo3w loading percent
max_loading_percent = list(net.trafo3w.max_loading_percent.values)
for i in range(len(max_loading_percent)):
    max_loading_percent[i] = 100.0
net.trafo3w.max_loading_percent = max_loading_percent

#Cost assignment

pp.create_pwl_costs(net, [i for i in range(len(net.gen.name.values))],et="gen", points=[[[0, 20, 1], [20, 30, 2]] for _ in range(len(net.gen.name.values))])
pp.create_pwl_costs(net, [i for i in range(len(net.sgen.name.values))],et="sgen", points=[[[0, 20, 0.25], [20, 30, 0.5]] for _ in range(len(net.sgen.name.values))])
pp.create_pwl_costs(net, [i for i in range(len(net.ext_grid.name.values))],et="ext_grid", points=[[[0, 20, 2], [20, 30, 5]] for _ in range(len(net.ext_grid.name.values))])

pp.runopp(net,verbose=True)
net.res_bus

In [ ]:
train_data, val_data, test_data = read_unsupervised_dataset('1-HV-mixed--0-no_sw')

In [ ]:
#train_data[0].has_isolated_nodes()
#train_data[0].has_self_loops()
#train_data[0].is_undirected()
x_dict = train_data[0].to_dict()
#to_json(train_dict)
ln = len(x_dict[("NB","-", "NB")]["edge_index"][0])
print(x_dict[("NB","-", "NB")]["edge_index"])#[:, :int(ln/2)]
x_dict

In [ ]:
net = sb.get_simbench_net('1-HV-mixed--0-no_sw')
idx_mapper, node_types_as_dict = extract_node_types_as_dict(net)
for key in node_types_as_dict:
    print(f"Bus Type: {key}")
    for i in range(len(node_types_as_dict[key])):
        #node_types_as_dict[key][i] = idx_mapper[node_types_as_dict[key][i]]
        print(str(node_types_as_dict[key][i]))
    print("-------------------------------")

In [ ]:
x_dict["PQ"]['x'][2]

In [ ]:
grids_available = []
for nw_name in all_simbench_codes:
        net = sb.get_simbench_net(nw_name)
        print("Trying Network named " + nw_name + "...")

        #dict_probs = pp.diagnostic(net,report_style='None')
        #for bus_num in dict_probs['multiple_voltage_controlling_elements_per_bus']['buses_with_gens_and_ext_grids']:
        #    net.gen = net.gen.drop(net.gen[net.gen.bus == bus_num].index)

        #OPERATIONAL CONSTRAINTS

        #Set upper and lower limits of active-reactive powers of loads
        min_p_mw_val, max_p_mw_val, min_q_mvar_val, max_q_mvar_val = [], [], [], []
        p_mw = list(net.load.p_mw.values)
        q_mvar = list(net.load.q_mvar.values)

        for i in range(len(p_mw)):
            min_p_mw_val.append(p_mw[i])
            max_p_mw_val.append(p_mw[i])
            min_q_mvar_val.append(q_mvar[i])
            max_q_mvar_val.append(q_mvar[i])

        net.load.min_p_mw = min_p_mw_val
        net.load.max_p_mw = max_p_mw_val
        net.load.min_q_mvar = min_q_mvar_val
        net.load.max_q_mvar = max_q_mvar_val

        #Replace all ext_grids but the first one with generators and set the generators to slack= false
        ext_grids = [i for i in range(1,len(net.ext_grid.name.values))]
        pp.replace_ext_grid_by_gen(net,ext_grids=ext_grids, slack=False)

        #TODO: reactive power limits for gens?


        #TODO: reactive power limits for sgens?



        #NETWORK CONSTRAINTS

        #Maximize the branch limits

        #max_i_ka = list(net.line.max_i_ka.values)

        #for i in range(len(max_i_ka)):
        # max_i_ka[i] = max(max_i_ka)



        #Maximize line loading percents
        max_loading_percent = list(net.line.max_loading_percent.values)
        for i in range(len(max_loading_percent)):
            max_loading_percent[i] = 100.0
        net.line.max_loading_percent = max_loading_percent

        #Maximize trafo loading percent
        max_loading_percent = list(net.trafo.max_loading_percent.values)
        for i in range(len(max_loading_percent)):
            max_loading_percent[i] = 100.0
        net.trafo.max_loading_percent = max_loading_percent

        #Maximize trafo3w loading percent
        max_loading_percent = list(net.trafo3w.max_loading_percent.values)
        for i in range(len(max_loading_percent)):
            max_loading_percent[i] = 100.0
        net.trafo3w.max_loading_percent = max_loading_percent

        #Cost assignment
        pp.create_pwl_costs(net, [i for i in range(len(net.gen.name.values))],et="gen", points=[[[0, 20, 1], [20, 30, 2]] for _ in range(len(net.gen.name.values))])
        pp.create_pwl_costs(net, [i for i in range(len(net.sgen.name.values))],et="sgen", points=[[[0, 20, 0.25], [20, 30, 0.5]] for _ in range(len(net.sgen.name.values))])
        pp.create_pwl_costs(net, [i for i in range(len(net.ext_grid.name.values))],et="ext_grid", points=[[[0, 20, 2], [20, 30, 5]] for _ in range(len(net.ext_grid.name.values))])

        try:
            pp.runpm_dc_opf(net) # Run DCOPP
        except pp.OPFNotConverged:
            text = "DC OPTIMAL POWERFLOW COMPUTATION DID NOT CONVERGE FOR NETWORK " + nw_name + ".SKIPPING THIS DATASET."
            print(text)
            continue
        print("GRID NAMED "+nw_name+" CONVERGES FOR DCOPF")
        grids_available.append(nw_name)


In [ ]:
for nw_name in grids_available:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) # dcopp on all grids directly

In [ ]:
grids_ready = []
for nw_name in all_simbench_codes[6:]:
    net = sb.get_simbench_net(nw_name)
    print("Trying Network named " + nw_name + "...")

    #OPERATIONAL CONSTRAINTS

    #Set upper and lower limits of active-reactive powers of loads
    min_p_mw_val, max_p_mw_val, min_q_mvar_val, max_q_mvar_val = [], [], [], []
    p_mw = list(net.load.p_mw.values)
    q_mvar = list(net.load.q_mvar.values)

    for i in range(len(p_mw)):
        min_p_mw_val.append(p_mw[i])
        max_p_mw_val.append(p_mw[i])
        min_q_mvar_val.append(q_mvar[i])
        max_q_mvar_val.append(q_mvar[i])

    net.load.min_p_mw = min_p_mw_val
    net.load.max_p_mw = max_p_mw_val
    net.load.min_q_mvar = min_q_mvar_val
    net.load.max_q_mvar = max_q_mvar_val

    #Replace all ext_grids but the first one with generators and set the generators to slack= false
    ext_grids = [i for i in range(1,len(net.ext_grid.name.values))]
    pp.replace_ext_grid_by_gen(net,ext_grids=ext_grids, slack=False)

    #TODO: reactive power limits for gens?


    #TODO: reactive power limits for sgens?



    #NETWORK CONSTRAINTS

    #Maximize the branch limits

    #max_i_ka = list(net.line.max_i_ka.values)

    #for i in range(len(max_i_ka)):
    # max_i_ka[i] = max(max_i_ka)



    #Maximize line loading percents
    max_loading_percent = list(net.line.max_loading_percent.values)
    for i in range(len(max_loading_percent)):
        max_loading_percent[i] = 100.0
    net.line.max_loading_percent = max_loading_percent

    #Maximize trafo loading percent
    max_loading_percent = list(net.trafo.max_loading_percent.values)
    for i in range(len(max_loading_percent)):
        max_loading_percent[i] = 100.0
    net.trafo.max_loading_percent = max_loading_percent

    #Maximize trafo3w loading percent
    max_loading_percent = list(net.trafo3w.max_loading_percent.values)
    for i in range(len(max_loading_percent)):
        max_loading_percent[i] = 100.0
    net.trafo3w.max_loading_percent = max_loading_percent

    #Cost assignment
    pp.create_pwl_costs(net, [i for i in range(len(net.gen.name.values))],et="gen", points=[[[0, 20, 1], [20, 30, 2]] for _ in range(len(net.gen.name.values))])
    pp.create_pwl_costs(net, [i for i in range(len(net.sgen.name.values))],et="sgen", points=[[[0, 20, 0.25], [20, 30, 0.5]] for _ in range(len(net.sgen.name.values))])
    pp.create_pwl_costs(net, [i for i in range(len(net.ext_grid.name.values))],et="ext_grid", points=[[[0, 20, 2], [20, 30, 5]] for _ in range(len(net.ext_grid.name.values))])

    #ac_converged = True

    #start_vec_name = ""
    #for init in ["pf", "flat", "results"]:
    #    try:
    #        pp.runopp(net, init=init)  # Calculate ACOPF with IPFOPT
    #    except pp.OPFNotConverged:
    #        if init == "results":
    #            text = "AC OPTIMAL POWERFLOW COMPUTATION DID NOT CONVERGE FOR NETWORK " + nw_name + ". SKIPPING THIS GRID."
    #            print(text)
    #            break
    #        continue
    #    start_vec_name = init
    #    ac_converged = True
    #    break
    #if ac_converged:
    #    print("GRID NAMED "+nw_name+" CONVERGES FOR ACOPF" + "WITH THE START VECTOR OPTION " + start_vec_name + ".")


    try:
        pp.runpm_ac_opf(net) # Run DCOPP
    except pp.OPFNotConverged:
        text = "AC OPTIMAL POWERFLOW COMPUTATION DID NOT CONVERGE FOR NETWORK " + nw_name + ".SKIPPING THIS DATASET."
        print(text)
        continue
    print("GRID NAMED "+nw_name+" CONVERGES FOR ACOPF" + "WITH THE START VECTOR OPTION " + ".")
    grids_ready.append(nw_name)


In [ ]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #julia acopf on all grids directly

In [ ]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #pp acopf on all grids directly

In [ ]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #julia acopf

In [ ]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #pp acopf

In [ ]:
dcopf_acopf_available_grid_names = ["1-HV-mixed--0-no_sw","1-HV-urban--0-no_sw", "1-MV-comm--0-no_sw", "1-MV-semiurb--0-no_sw"]

In [ ]:
datasets_df_as_list = []
for grid_name in dcopf_acopf_available_grid_names:
    print("Generating datasets for grid " + grid_name)
    while len(datasets_df_as_list) != 100:
        net = sb.get_simbench_net(grid_name)
        df = create_dataset_from_dcopf_and_acopf(net)
        if df is not None:
            datasets_df_as_list.append(df)
            print(str(len(datasets_df_as_list)) + " Dataset(s) Generated" + " for the grid " + grid_name)

    print("Saving the datasets for grid " + grid_name)
    path = os.path.dirname(os.path.abspath("gnn.ipynb")) + "\\data\\Supervised\\Training\\" + grid_name
    if not os.path.exists(path):
        os.makedirs(path)

    for i in range(0, len(datasets_df_as_list)):
        newpath = path + "\\Dataset-" + str(i) + ".csv"
        datasets_df_as_list[i].to_csv(newpath)

    datasets_df_as_list.clear()

In [ ]:
dcopf_available_grid_names = ["1-HVMV-mixed-all-0-no_sw", "1-HVMV-mixed-1.105-0-no_sw", "1-HVMV-mixed-2.102-0-no_sw","1-HVMV-mixed-4.101-0-no_sw", "1-HVMV-urban-all-0-no_sw", "1-HVMV-urban-2.203-0-no_sw", "1-HVMV-urban-3.201-0-no_sw", "1-HVMV-urban-4.201-0-no_sw", "1-HV-mixed--0-no_sw", "1-HV-urban--0-no_sw", "1-MVLV-rural-all-0-no_sw"]

In [ ]:
datasets_df_as_list = []
for grid_name in dcopf_available_grid_names:
    print("Generating datasets for grid " + grid_name)
    while len(datasets_df_as_list) != 100:
        net = sb.get_simbench_net(grid_name)
        df = create_dataset_from_dcopf(net)
        if df is not None:
            datasets_df_as_list.append(df)
            print(str(len(datasets_df_as_list)) + " Dataset(s) Generated" + " for the grid " + grid_name)

    print("Saving the datasets for grid " + grid_name)
    path = os.path.dirname(os.path.abspath("gnn.ipynb")) + "\\data\\Unsupervised\\Training\\" + grid_name
    if not os.path.exists(path):
        os.makedirs(path)

    for i in range(0, len(datasets_df_as_list)):
        newpath = path + "\\Dataset-" + str(i) + ".csv"
        datasets_df_as_list[i].to_csv(newpath)

    datasets_df_as_list.clear()

In this revised version of the compute_node_embeddings function, the node features are weighted by the reverse admittance values in the adjacency matrix before they are summed to compute the node embeddings. The resulting node embeddings will reflect the strength of the connections between the nodes.

To use the reverse admittance values as the edge weights, you would need to pass the Ybus matrix as the adjacency matrix when calling the compute_node_embeddings function. The Ybus matrix should be converted to a PyTorch tensor before passing it to the function.

use the reverse admittance values as edge weights, you can modify the computation of the node embeddings to weight the node features by the reverse admittance values.

# Define the node types
node_types = ['Slack Node', 'Generator Node', 'Load Node']

# Define the number of nodes of each type in the graph
num_nodes = {
    'Slack Node': 1,
    'Generator Node': 20,
    'Load Node': 99
}


In [ ]:
grid_names = [_ for _ in os.listdir(os.path.dirname(os.path.abspath("gnn.ipynb")) + "\\data\\Supervised\\")]


In [ ]:
graphdata_lst = read_multiple_supervised_datasets(grid_names)

In [5]:
in_channels = 4
hidden_channels = 256
out_channels = 4
activation = "elu"
scaler = StandardScaler()
num_layers = 5
dropout = 0.0
jk = "last"
lr = 0.00001
layer_type = "TransConv"
# torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels)
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN(4, 256, num_layers, 4, dropout=0.0, norm=torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels),jk="last",layer_type="TransConv", activation="elu")#.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

ordered_dict = torch.load(r"C:\Users\canba\OneDrive\Masaüstü\OPFGNN\code\Models\Supervised\supervisedmodel.pt")

model.load_state_dict(ordered_dict)

<All keys matched successfully>

In [ ]:
lr

In [ ]:
test_datasets_lst = []
for i, graphdata in enumerate(graphdata_lst):
    run = wandb.init(
    # set the wandb project where this run will be logged
    project="OPF-GNN-Supervised-Homo",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "Homo-GNN",
    "num_layers": num_layers,
    "dataset": grid_names[i],
    "epochs": 1000,
    "activation": activation,
    "dropout": dropout,
    "in_channels": in_channels,
    "output_channels": out_channels,
    "hidden_channels": hidden_channels,
    "channel type": layer_type,
    "norm": "BatchNorm",
    "scaler": scaler

    }
    )
    num_epochs = wandb.run.config.epochs
    run.watch(model)
    grid_name = graphdata.grid_name
    run.config.dataset = grid_name
    train_data = graphdata.train_data
    run.config["number of busses"] = np.shape(train_data[0].x)[0]
    val_data = graphdata.val_data
    test_data = graphdata.test_data
    test_datasets_lst.append(test_data)
    training_loader = DataLoader(train_data, batch_size=1, shuffle=True)
    validation_loader = DataLoader(val_data, batch_size=1, shuffle=True)
    #test_loader = DataLoader(test_data, batch_size=1, shuffle=True)
    for _ in range(num_epochs):
        #train_one_epoch(i, optimizer, training_loader, model, nn.MSELoss(), edge_index, edge_weights)
        train_validate_one_epoch(_, grid_name, optimizer, training_loader, validation_loader, model, nn.MSELoss(), scaler)
    print("Training and Validation finished " + "for GraphData " + str(i) + ".")


In [ ]:
train_loader_lst = []
val_loader_lst = []
test_loader_lst = []
for i, graphdata in enumerate(graphdata_lst):

    # Divide training data into chunks, load into Dataloaders and append to the list of training loaders
    for train_data in divide_chunks(graphdata.train_data, 5):
        train_loader_lst.append(DataLoader(train_data, batch_size=1, shuffle=True))

    # Divide validation data into chunks, load into Dataloaders and append to the list of validation loaders
    for val_data in divide_chunks(graphdata.val_data, 5):
        val_loader_lst.append(DataLoader(val_data, batch_size=1, shuffle=True))

    # Append the test data to the list
    for test_data in divide_chunks(graphdata.test_data, 5):
        test_loader_lst.append(DataLoader(test_data, batch_size=1, shuffle=True))

print("Data Preparation finished.")

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr/100)

In [ ]:
run = wandb.init(
    # set the wandb project where this run will be logged
    project="OPF-GNN-Supervised-Homo",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "Homo-GNN",
    "num_layers": num_layers,
    "epochs": 2000,
    "activation": activation,
    "dropout": dropout,
    "in_channels": in_channels,
    "output_channels": out_channels,
    "hidden_channels": hidden_channels,
    "channel type": layer_type,
    "norm": "BatchNorm",
    "scaler": scaler
    }
    )
"""
for _ in range(wandb.run.config.epochs):
    # Training
    random.shuffle(train_loader_lst)
    print("Training the model for epoch " + str(_))
    train_all_one_epoch(_, optimizer, train_loader_lst, model, nn.MSELoss())

    # Validation
    random.shuffle(val_loader_lst)
    print("Validating the model for epoch " + str(_))
    validate_all_one_epoch(_, val_loader_lst, model, nn.MSELoss())

    outputs = test_all_one_epoch(test_loader_lst, model, nn.MSELoss())
print("Training and Validation finished.")
"""


In [ ]:
outputs = test_all_one_epoch(test_loader_lst, model, nn.MSELoss())

In [ ]:
output,target = outputs[0]

In [ ]:
path = r"C:\Users\canba\OneDrive\Masaüstü\OPFGNN\code\Models\Supervised\supervisedmodel.pt" #os.path.dirname(os.path.abspath("gnn.ipynb")) + "\\Models\\Supervised\\" + "basemodel.pt" #r"C:\Users\canba\OneDrive\Masaüstü\OPFGNN\code\Models\Supervised"
torch.save(model.state_dict(), "supervisedmodel.pt")
print("done")



In [ ]:
'/'.join(os.path.dirname(os.path.abspath("gnn.ipynb")).split("\\"))+ r"/Models/SelfSupervised/base_model.pt"

In [ ]:
from pandapower.plotting.simple_plot import simple_plot
from pandapower.plotting.plotly.simple_plotly import simple_plotly
#simple_plot(net, plot_gens=True, plot_loads=True, plot_sgens=True, library="igraph")
simple_plotly(net, map_style="satellite")

In [ ]:
pp.runpm_ac_opf(net)
net.res_bus

In [ ]:
pp.runpm_ac_opf(net)
net.res_bus

In [ ]:
pp.runopp(net)
net.res_bus

In [ ]:
idx_mapper, node_types_as_dict = extract_node_types_as_dict(net)
node_types_as_dict

In [4]:
in_channels = 4
hidden_channels = 256
out_channels = 4
activation = "elu"
scaler = StandardScaler()
num_layers = 5
dropout = 0.0
jk = "last"
lr = 0.00001
layer_type = "TransConv"
# torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels)
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN(in_channels, hidden_channels, num_layers, out_channels, dropout=dropout, norm=torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels),jk=jk,layer_type=layer_type, activation=activation)#.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

ordered_dict = torch.load(r"C:\Users\canba\OneDrive\Masaüstü\OPFGNN\code\Models\Supervised\supervisedmodel.pt")

model.load_state_dict(ordered_dict)

<All keys matched successfully>

In [5]:
model.parameters

<bound method Module.parameters of GNN(4, 4, num_layers=5)>

In [ ]:
grid_name = '1-HV-mixed--0-no_sw'

In [ ]:
pp.runpm_ac_opf(net)


In [ ]:
grid_name = '1-HV-mixed--0-no_sw'
net = process_network(grid_name)


In [ ]:
pp.runpm_ac_opf(net)
net.res_bus

 HETEROGENEOUS GNN

In [2]:
index_mappers,net,data = generate_unsupervised_input('1-HV-mixed--0-no_sw')

In [ ]:
model = create_ACOPFGNN_model(data)

In [3]:
x_dict, constraint_dict, _, _, bus_idx_neighbors_dict,scalers_dict, _ = extract_unsupervised_inputs(data)

In [4]:
x_dict

{'SB': tensor([[ 0.7618, -1.0000,  0.0000,  0.0000]], requires_grad=True),
 'PQ': tensor([[-0.9310,  0.0000,  0.0091,  0.0037],
         [-0.9310, -0.6667,  0.0094,  0.0034],
         [-0.9310,  0.1667,  0.0538,  0.0260],
         [-0.9310, -0.5000,  0.1295,  0.0438],
         [-0.9310,  0.1667,  0.0090,  0.0032],
         [-0.9310,  0.0000,  0.0083,  0.0033],
         [-0.9310,  0.5000,  0.1223,  0.0493],
         [-0.9310, -0.1667,  0.0806,  0.0232],
         [-0.9310, -0.1667,  0.0093,  0.0031],
         [-0.9310, -0.3333,  0.0084,  0.0035],
         [-0.9310,  1.0000,  0.0083,  0.0035],
         [-0.9310,  0.0000,  0.0562,  0.0209],
         [-0.9310,  0.0000,  0.0612,  0.0229],
         [-0.9310,  0.0000,  0.0083,  0.0031],
         [-0.9310, -0.5000,  0.0085,  0.0036],
         [-0.9310,  0.3333,  0.0088,  0.0036]], requires_grad=True),
 'PV': tensor([[ 0.7618, -0.1667, -0.5000, -0.5000],
         [-0.2414,  0.0000, -0.4286, -0.4286],
         [-0.9310,  0.6667, -0.0122, -0.0048]

In [ ]:
constraint_dict

In [ ]:
ACOPFOutput(x_dict, scalers_dict, None, index_mappers).output

In [ ]:
grid_names = sb.collect_all_simbench_codes()[25:]
acopf_ok_grid_names = []

for grid_name in grid_names:
    print(f"Trying grid {grid_name}")
    net = process_network(grid_name)

    # try:
    #     acopf_ok_grid_names.append(grid_name)
    #     pp.runpm_ac_opf(net)
    #     #print(net.res_bus)
    # except:
    #     print(f"Julia didnt converge for {grid_name}")
    #     acopf_ok_grid_names.remove(grid_name)
    try:
        acopf_ok_grid_names.append(grid_name)
        pp.runopp(net)
        #print(net.res_bus)
    except:
        print(f"OPP didnt converge for {grid_name}")
        acopf_ok_grid_names.remove(grid_name)
acopf_ok_grid_names
#save_multiple_unsupervised_inputs(grid_names, 100)

In [15]:
acopf_ok_grid_names = ['1-HV-mixed--1-sw','1-MV-semiurb--0-no_sw','1-MV-comm--0-sw','1-MV-comm--0-no_sw']
net = process_network(acopf_ok_grid_names[1])
pp.runopp(net)
net.res_bus

no costs are given - overall generated power is minimized


,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.025000,0.000000,-8.057173,-11.835759,1.000000,2.259515e-21
2,1.001425,209.119600,-1.176327,0.135443,1.000714,1.887024e-03
4,0.999942,209.185313,0.397800,0.183689,1.002495,4.950700e-03
5,0.999154,209.227646,0.293150,0.148678,1.003357,6.811706e-03
6,0.998564,209.265084,0.239906,0.171257,1.003935,8.382988e-03
...,...,...,...,...,...,...
112,0.987459,209.245271,0.237492,0.096853,1.020719,1.774877e-02
113,0.987333,209.251486,0.036984,0.085200,1.020904,1.802879e-02
114,0.987261,209.254787,-0.074431,0.034476,1.021013,1.818089e-02
115,0.987149,209.258063,0.118649,0.137547,1.021198,1.835478e-02


In [ ]:
acopf_grid_names = ['1-HV-mixed--0-no_sw','1-MV-semiurb--0-no_sw', '1-MV-comm--0-no_sw']
save_multiple_unsupervised_inputs(acopf_grid_names, num_samples=100)

In [4]:
inputs = load_multiple_unsupervised_inputs()
for i,input in enumerate(inputs):
    if input.res_bus is None:
        print(f"None at {i}")

In [ ]:
save_unsupervised_inputs('1-HV-mixed--0-no_sw', 100)

In [3]:
inputs = load_unsupervised_inputs('1-HV-mixed--0-no_sw')
inputs[0].res_bus

{'SB': tensor([[ 1., -1.,  1., -1.]]),
 'PQ': tensor([[-0.9873,  0.7709, -0.3623, -0.1186],
         [-0.9394, -0.0805, -0.3617, -0.1180],
         [-0.9398, -0.0860, -0.3399, -0.1061],
         [-0.9422, -0.1190, -0.2339, -0.0596],
         [-0.8992,  0.4338, -0.3625, -0.1181],
         [-0.9888,  0.5735, -0.3609, -0.1185],
         [-0.9211,  0.1659, -0.2409, -0.0530],
         [-0.9576,  0.1665, -0.3611, -0.1181],
         [-0.9398, -0.0755, -0.3623, -0.1183],
         [-0.9895,  0.5856, -0.3613, -0.1186],
         [-0.9341,  0.2072, -0.3541, -0.1106],
         [-0.9940,  0.7561, -0.3464, -0.1047],
         [-0.9886,  0.5763, -0.3623, -0.1183],
         [-0.9395, -0.0816, -0.3612, -0.1184],
         [-0.9347,  0.7639, -0.3609, -0.1177]]),
 'PV': tensor([[ 0.5288, -0.2341,  0.6834,  0.5512],
         [-0.4013,  0.3430,  0.2802,  1.0000],
         [-1.0000,  0.7024, -0.3831, -0.1285],
         [-0.9931,  0.7434, -0.4373, -0.1552],
         [-0.9809,  0.8055, -0.3962, -0.1350],
       

In [ ]:
inputs

In [ ]:
inputs[57].res_bus

Display Plot of Customized Sigmoid Function

In [ ]:
import matplotlib.pyplot as p
import torch

def custom_tanh(x: torch.Tensor, lower_bound: float, upper_bound: float) -> torch.Tensor:
    width = upper_bound - lower_bound
    return 0.5 * width * torch.tanh(x) + 0.5 * (upper_bound + lower_bound)


lst = []
min_val = 0
max_val = 0.1
range_ = []
i = -2
while i < 2:
    range_.append(i)
    i += 0.01

for i in range_:
    val = custom_tanh(torch.tensor(i), min_val, max_val)
    #val = torch.tanh(torch.tensor(i))
    lst.append(val)

# Now, plot the values in lst
p.plot(range_, lst)
p.xlabel('Input')
p.ylabel('Value')
p.title('Plot of Enforcing Activation Function')
p.grid(True)
p.show()
print(range_)

Create Heterogeneous Self Supervised Model and Process Inputs

In [2]:
# Define the Parameters
grid_name = '1-HV-mixed--0-no_sw'


In [ ]:
# Load the Model
model = load_ACOPFGNN_model(grid_name, "base_model.pt", hidden_channels=128, num_layers=3)

In [3]:
# Create ACOPFGNN model and Optimizer
index_mappers,net,data = generate_unsupervised_input(grid_name)
model = create_ACOPFGNN_model(data, net, index_mappers, hidden_channels=32, num_layers=3)

no costs are given - overall generated power is minimized
no costs are given - overall generated power is minimized


defaultdict(<class 'list'>, {'SB': tensor([[ 1.0410,  0.1195,  0.1117, -0.5527]], grad_fn=<EluBackward0>), 'PQ': tensor([[ 0.7845,  0.5525, -0.3123,  0.9621],
        [ 0.9402, -0.0822, -0.5976,  1.0648],
        [ 0.7305,  0.6192, -0.2504,  0.9214],
        [ 0.6637,  0.6712, -0.1704,  0.8417],
        [ 0.9081, -0.0444, -0.5810,  1.0999],
        [ 0.3656,  0.6597, -0.0082,  0.7576],
        [ 0.7483, -0.3726, -0.6642,  1.1994],
        [ 0.8414, -0.0771, -0.5779,  1.1349],
        [ 0.3199,  0.6072, -0.0699,  0.8908],
        [ 1.1162, -0.1804, -0.5681,  0.6163],
        [ 0.4061, -0.5692, -0.7087,  1.3860],
        [ 0.6826,  0.6575, -0.1870,  0.8458],
        [ 0.8687,  0.3222, -0.4489,  1.0479],
        [ 0.0871,  0.1304, -0.2140,  1.0187],
        [ 0.9251,  0.5312, -0.3394,  0.9130],
        [ 0.7109,  0.6334, -0.2208,  0.8751]], grad_fn=<EluBackward0>), 'PV': tensor([[-0.3689,  0.6374,  0.7594, -0.0317],
        [-0.4046,  0.5157,  0.7236, -0.0487],
        [-0.6544,  0.7219, 

In [4]:
num_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
#lr_per_mil_param = 1e-5/1000000
learning_rate = 2.5e-4#(num_parameters/1000000) * lr_per_mil_param
optimizer = torch.optim.Adam(model.parameters())

In [5]:
num_parameters

23705

In [6]:
WANDB_NOTEBOOK_NAME = "msi"
# Initialize Weights & Biases
run = wandb.init(
        # set the wandb project where this run will be logged
        project="ACOPF-GNN-SelfSupervised-Hetero",

        # track hyperparameters and run metadata
        config={
            #"learning_rate": optimizer.,
            "architecture": "Hetero-SelfSupervised-GNN",
            "num_parameters": sum(p.numel() for p in model.parameters() if p.requires_grad),
            "num_heads": model.heads,
            "num_layers": model.num_layers,
            "grid_name": grid_name,
            "activation": model.act_fn,
            "dropout": model.dropout,
            "in_channels": model.in_channels,
            "output_channels": model.out_channels,
            "hidden_channels": model.hidden_channels,
            "channel type": "TransformerConv",
            "scaler": "MinMax",
            "model": "base_model_wired"
        }
    )


wandb: Currently logged in as: canbay96 (cbml). Use `wandb login --relogin` to force relogin


In [7]:
# Load Inputs
inputs = load_unsupervised_inputs(grid_name)
#inputs = load_multiple_unsupervised_inputs()
n = len(inputs)
train_inputs = inputs[:int(n*0.8)]
val_inputs = inputs[int(n*0.8): int(n*0.95)]
test_inputs = inputs[int(n*0.95):]



In [8]:

# Train and Validate Model
start_epoch = 1
num_epochs = 20
loss_weights = (0.0, 1.0, 0.0)
model_name = "base_model_physics.pt"
train_output, val_output = train_validate_ACOPF(model, model_name, optimizer, train_inputs, val_inputs,loss_weights, start_epoch, num_epochs)

---------------   CURRENT LEARNING RATE: 0.001   ---------------
Epoch: 0
###########################################
   TRAINING
     Training Step: 0 Training Loss: 1.9385786056518555 
     Training Step: 1 Training Loss: 1.493556022644043 
     Training Step: 2 Training Loss: 1.2006518840789795 
     Training Step: 3 Training Loss: 1.0047500133514404 
     Training Step: 4 Training Loss: 0.8729242086410522 
     Training Step: 5 Training Loss: 0.7745358943939209 
     Training Step: 6 Training Loss: 0.7127636671066284 
     Training Step: 7 Training Loss: 0.6239981651306152 
     Training Step: 8 Training Loss: 0.588146984577179 
     Training Step: 9 Training Loss: 0.5374649167060852 
     Training Step: 10 Training Loss: 0.47320985794067383 
     Training Step: 11 Training Loss: 0.4327434301376343 
     Training Step: 12 Training Loss: 0.41743409633636475 
     Training Step: 13 Training Loss: 0.3629176616668701 
     Training Step: 14 Training Loss: 0.33380356431007385 
     Trai

In [11]:
for i in range(1, 100, 20) :
    start_epoch = i
    num_epochs = 20 + i
    model_name = "base_model_wired_alt.pt"
    train_output, val_output = train_validate_ACOPF(model, "base_model.pt", optimizer, train_inputs, val_inputs, start_epoch, num_epochs)

---------------   CURRENT LEARNING RATE: 0.001   ---------------
Epoch: 0
###########################################
   TRAINING
     Training Step: 0 Training Loss: 2294.640380859375 
     Training Step: 1 Training Loss: 1887.6961669921875 
     Training Step: 2 Training Loss: 1599.8555908203125 
     Training Step: 3 Training Loss: 1371.1243896484375 
     Training Step: 4 Training Loss: 1237.79150390625 
     Training Step: 5 Training Loss: 1108.6446533203125 
     Training Step: 6 Training Loss: 981.4968872070312 
     Training Step: 7 Training Loss: 887.6901245117188 
     Training Step: 8 Training Loss: 803.025390625 
     Training Step: 9 Training Loss: 737.6721801757812 
     Training Step: 10 Training Loss: 655.5064697265625 
     Training Step: 11 Training Loss: 602.1118774414062 
     Training Step: 12 Training Loss: 544.0517578125 
     Training Step: 13 Training Loss: 482.4895935058594 
     Training Step: 14 Training Loss: 444.9730224609375 
     Training Step: 15 Traini

KeyboardInterrupt: 

In [9]:
train_output.output

array([[  0.65535082,   0.85065937, 101.1341095 ,  30.60231972],
       [  0.87852446,   0.        ,  97.71369934,  26.49389076],
       [  1.17149561,   0.52058458,  91.36586761,  25.2800045 ],
       [  2.35825972,   1.26082039,  98.54285431,  28.40731621],
       [  2.47807118,   1.90889263,  96.10548401,  26.99606323],
       [  2.32928439,   2.10406399,  98.14118958,  18.67367935],
       [  2.29653556,   0.48540926,  95.84825897,  23.06588745],
       [  2.27876434,   0.65570354,  97.94632721,  28.5507164 ],
       [  2.23215776,   0.57320786,  98.65805054,  28.91212845],
       [  2.49235646,   1.88243008,  96.47057343,  26.99268532],
       [  2.54828547,   0.45856953,  97.71361542,  26.83773232],
       [  2.24220165,   0.53578043,  97.63864136,  27.81454277],
       [  2.4400496 ,   0.45287943,  95.73828125,  27.18199539],
       [  2.33850514,   0.54629946,  97.04912567,  28.78719521],
       [  2.36899636,   1.25119305,  97.65576935,  27.05794525],
       [  2.50876187,   1

In [10]:
val_output.output

array([[  0.65395636,   0.91567945, 113.43726349,  29.42810631],
       [  0.87851281,   0.        , 108.61277008,  25.59692192],
       [  1.16987846,   0.55724096, 103.16665649,  23.53556824],
       [  2.36202032,   1.31174994, 109.1506958 ,  26.64596558],
       [  2.47899891,   2.01026392, 107.45316315,  25.36096764],
       [  2.32930631,   2.2199378 , 109.58473206,  16.79294968],
       [  2.29702703,   0.51201105, 107.33950043,  21.24329758],
       [  2.2801909 ,   0.68483496, 109.45345306,  26.64138985],
       [  2.23273204,   0.60182142, 110.2062912 ,  27.19800758],
       [  2.49354581,   1.98105192, 107.84737396,  25.28386307],
       [  2.54720293,   0.48814821, 109.29827118,  25.25607681],
       [  2.24280396,   0.56217146, 109.1360321 ,  26.07968903],
       [  2.43797025,   0.48188353, 107.30545044,  25.60661697],
       [  2.33934659,   0.57242441, 108.4782486 ,  26.97213173],
       [  2.37285434,   1.30195093, 108.24641418,  25.27768898],
       [  2.50968989,   1

In [10]:
print(sum(train_output.output[:, 2]), sum(train_output.output[:, 3]))
train_output.output

5966.011412382126 1892.8524532318115


array([[ 7.15626927e-01,  1.60224915e-01,  1.15818100e+02,
         3.00056400e+01],
       [ 9.07988217e-01,  0.00000000e+00,  9.94839325e+01,
        -1.02789154e+02],
       [ 1.23604334e+00,  1.46118164e-01,  1.15757896e+02,
         3.01666393e+01],
       [ 2.55935558e+00, -2.05223799e+00,  3.12666154e+00,
         2.69802341e+01],
       [ 2.44898793e+00, -3.17946029e+00,  4.55197945e+01,
         1.82022038e+01],
       [ 2.69901456e+00, -3.27771020e+00,  1.27916498e+01,
         1.49252577e+01],
       [ 2.45637790e+00,  6.56132698e-02,  1.17606575e+02,
         3.24012260e+01],
       [ 2.46242343e+00,  1.08196735e-01,  1.16958267e+02,
         3.13654480e+01],
       [ 2.45159524e+00,  6.78076744e-02,  1.19253365e+02,
         3.20844154e+01],
       [ 2.44778803e+00, -3.13636684e+00,  4.60496750e+01,
         1.50414839e+01],
       [ 2.33369640e+00, -6.41553402e-01,  1.32012894e+02,
         5.11921272e+01],
       [ 2.45274991e+00,  6.70785904e-02,  1.19010818e+02,
      

In [11]:
print(sum(val_output.output[:, 2]), sum(val_output.output[:, 3]))
val_output.output

6574.148982524872 1890.5028705596924


array([[ 7.15729081e-01,  1.77783489e-01,  1.26331802e+02,
         3.00175591e+01],
       [ 9.08043390e-01,  0.00000000e+00,  1.09311668e+02,
        -1.04568199e+02],
       [ 1.23621632e+00,  1.62799835e-01,  1.26268112e+02,
         3.01858730e+01],
       [ 2.55875272e+00, -2.13988113e+00,  9.35225296e+00,
         2.72275620e+01],
       [ 2.45310530e+00, -3.31060219e+00,  5.29221764e+01,
         1.79087620e+01],
       [ 2.70498879e+00, -3.41292596e+00,  1.88522415e+01,
         1.44304237e+01],
       [ 2.45709201e+00,  7.98630714e-02,  1.28134369e+02,
         3.24117546e+01],
       [ 2.46299605e+00,  1.22897625e-01,  1.27475647e+02,
         3.13910370e+01],
       [ 2.45226995e+00,  8.12582970e-02,  1.29818588e+02,
         3.21212082e+01],
       [ 2.45450107e+00, -3.26728702e+00,  5.30583572e+01,
         1.46673098e+01],
       [ 2.33528609e+00, -6.52376175e-01,  1.42961456e+02,
         5.13160248e+01],
       [ 2.45337885e+00,  8.06016922e-02,  1.29578094e+02,
      

In [ ]:
train_output.res_bus

In [11]:
def save_ACOPFGNN_model(model: ACOPFGNN):
    path = r"./Models/SelfSupervised/base_model_physics.pt"

    torch.save(model.state_dict(), path)
save_ACOPFGNN_model(model)

In [ ]:
train_output.output

In [9]:
optimizer.param_groups[0]['lr'] /= 2.5
optimizer.param_groups[0]['lr']

1e-05

In [10]:
wandb.finish()

In [ ]:
# Train and Validate Model
start_epoch = 101
num_epochs = 150

train_output, val_output = train_validate_ACOPF(model, optimizer, train_inputs, val_inputs, start_epoch, num_epochs)